# Assignment 2

In [ ]:
import cv2 as cv
import numpy as np
import PIL.Image as Image
from IPython.display import clear_output, Markdown

corridor = [cv.imread('corridorl.jpg', cv.IMREAD_GRAYSCALE), cv.imread('corridorr.jpg', cv.IMREAD_GRAYSCALE)]
triclopsi2 = [cv.imread('triclopsi2l.jpg', cv.IMREAD_GRAYSCALE), cv.imread('triclopsi2r.jpg', cv.IMREAD_GRAYSCALE)]

In [ ]:
def computeDisparity(l, r, numDisparities, blockSize):
  N = int((blockSize / 2))
  lPadded = np.pad(l, N).astype(np.int)
  rPadded = np.pad(r, N).astype(np.int)
  lOut = np.zeros(l.shape, np.uint8)
  rOut = np.zeros(r.shape, np.uint8)
  lCrpd = np.zeros(l.shape, np.uint8)
  for y in range(l.shape[0]):
    if y % 10 == 0:
      clear_output(True)
      print('Progress: %d%%' % int(y/l.shape[0] * 100))

    for lx in range(numDisparities, l.shape[1]):
      # Extract block from left image
      lBlock = np.array(lPadded[y:y+blockSize,lx:lx+blockSize])

      # Extract blocks from right image for image correlation
      # Assume no negative disparity and only search within range left of left image pixel specified by numDisparities(maximum disparity)
      rBlocks = np.array([rPadded[np.arange(y,y+blockSize), rx:rx+blockSize] for rx in range(lx-numDisparities, lx+1)])

      # Batch apply sum of squared differences for all right image pixels with left image pixel
      similarity = np.array(np.sum((rBlocks-lBlock)**2, (1,2)))

      # Compute disparity and matching right pixel coordinate
      similarity = np.flip(similarity)
      disparity = similarity.argmin()
      #disparity = numDisparities - similarity.argmin()
      rx = lx - disparity

      # Normalize disparity to 8-bit range
      nDisparity = np.uint8(disparity / numDisparities * 255)

      lOut[y, lx] = nDisparity
      rOut[y, rx] = nDisparity
      lCrpd[y, lx] = rx

  clear_output(True)
  print('Progress: 100%')
  return [lOut, rOut], lCrpd

def computeDisparityRight(l, r, numDisparities, blockSize):
  N = int((blockSize / 2))
  lPadded = np.pad(l, N).astype(np.int)
  rPadded = np.pad(r, N).astype(np.int)
  lOut = np.zeros(l.shape, np.uint8)
  rOut = np.zeros(r.shape, np.uint8)
  rCrpd = np.zeros(r.shape, np.uint8)
  for y in range(l.shape[0]):
    if y % 10 == 0:
      clear_output(True)
      print('Progress: %d%%' % int(y/r.shape[0] * 100))

    for rx in range(0, r.shape[1] - numDisparities):
      # Extract block from right image
      rBlock = np.array(rPadded[y:y+blockSize,rx:rx+blockSize])

      # Extract blocks from left image for image correlation
      # Assume no negative disparity and only search within range right of right image pixel specified by numDisparities(maximum disparity)
      lBlocks = np.array([lPadded[np.arange(y,y+blockSize), lx:lx+blockSize] for lx in range(rx, rx+numDisparities+1)])

      # Batch apply sum of squared differences for all left image pixels with right image pixel
      similarity = np.array(np.sum((lBlocks-rBlock)**2, (1,2)))

      # Compute disparity and matching right pixel coordinate
      disparity = similarity.argmin()
      lx = rx + disparity

      # Normalize disparity to 8-bit range
      nDisparity = np.uint8(disparity / numDisparities * 255)

      lOut[y, lx] = nDisparity
      rOut[y, rx] = nDisparity
      rCrpd[y, rx] = lx

  clear_output(True)
  print('Progress: 100%')
  return [lOut, rOut], rCrpd

def occlusion_check(lcrpd, rcrpd, dp, numDisparities):
  tmp_dp = dp.astype(np.uint16)
  # If right corresponding pixel in left disparity map does not match left corresponding pixel in right disparity map, invalidate computed disparity
  for y, lrow in enumerate(lcrpd):
    for lx in range(numDisparities, len(lrow)):
      if rcrpd[y, lrow[lx]] != lx:
        tmp_dp[y, lx] = 256

  # For all invalidated disparities, replace with the smallest neighbouring disparity i.e. disparity of background
  for y, lrow in enumerate(tmp_dp):
    for lx in range(numDisparities, len(lrow)):
      if lrow[lx] == 256:
        valid_dp = np.array([256, 256], np.int16)

        idx = lx-1
        while valid_dp[0] == 256 and idx >= 0:
          if lrow[idx] != 256:
            valid_dp[0] = lrow[idx]
          idx -= 1

        idx = lx+1
        while valid_dp[1] == 256 and idx < len(lrow):
          if lrow[idx] != 256:
            valid_dp[1] = lrow[idx]
          idx += 1

        lrow[lx] = valid_dp.min()

  return tmp_dp.astype(np.uint8)

## Corridor

In [ ]:
numDisparities = 10
blockSize = 3

corridor_PIL = [Image.fromarray(dp) for dp in corridor]
display(Markdown('### Left View'))
display(corridor_PIL[0])
display(Markdown('### Right View'))
display(corridor_PIL[1])

In [ ]:
corridor_dp_l, corridor_lcrpd = computeDisparity(corridor[0], corridor[1], numDisparities, blockSize)
corridor_PIL = [Image.fromarray(dp) for dp in corridor_dp_l]
display(Markdown('### Left Disparity Map'))
display(corridor_PIL[0])
display(Markdown('### Right Correspondence'))
display(corridor_PIL[1])

In [ ]:
corridor_dp_r, corridor_rcrpd = computeDisparityRight(corridor[0], corridor[1], numDisparities, blockSize)
corridor_PIL = [Image.fromarray(dp) for dp in corridor_dp_r]
display(Markdown('### Left Correspondence'))
display(corridor_PIL[0])
display(Markdown('### Right Disparity Map'))
display(corridor_PIL[1])

In [ ]:
corridor_dp = [corridor_dp_l[0], corridor_dp_r[1]]
corridor_PIL = [Image.fromarray(dp) for dp in corridor_dp]
display(Markdown('### Left Disparity Map'))
display(corridor_PIL[0])
display(Markdown('### Right Disparity Map'))
display(corridor_PIL[1])

In [ ]:
occlude_dp = occlusion_check(corridor_lcrpd, corridor_rcrpd, corridor_dp[0], numDisparities)
occlude_dp_filtered = cv.medianBlur(occlude_dp, 3)

display(Markdown('### Original Disparity Map'))
display(Image.fromarray(corridor_dp[0]))
display(Markdown('### Occlusion Filling'))
display(Image.fromarray(occlude_dp))
display(Markdown('### Occlusion Filling + Median Filtering'))
display(Image.fromarray(occlude_dp_filtered))

## Triclopsi2

In [ ]:
numDisparities = 10
blockSize = 16

triclopsi2_PIL = [Image.fromarray(dp) for dp in triclopsi2]
display(Markdown('### Left View'))
display(triclopsi2_PIL[0])
display(Markdown('### Right View'))
display(triclopsi2_PIL[1])

In [ ]:
triclopsi2_dp_l, triclopsi2_lcrpd = computeDisparity(triclopsi2[0], triclopsi2[1], numDisparities, blockSize)
triclopsi2_PIL = [Image.fromarray(dp) for dp in triclopsi2_dp_l]
display(Markdown('### Left Disparity Map'))
display(triclopsi2_PIL[0])
display(Markdown('### Right Correspondence'))
display(triclopsi2_PIL[1])

In [ ]:
triclopsi2_dp_r, triclopsi2_rcrpd = computeDisparityRight(triclopsi2[0], triclopsi2[1], numDisparities, blockSize)
triclopsi2_PIL = [Image.fromarray(dp) for dp in triclopsi2_dp_r]
display(Markdown('### Left Correspondence'))
display(triclopsi2_PIL[0])
display(Markdown('### Right Disparity Map'))
display(triclopsi2_PIL[1])

In [ ]:
triclopsi2_dp = [triclopsi2_dp_l[0], triclopsi2_dp_r[1]]
triclopsi2_PIL = [Image.fromarray(dp) for dp in triclopsi2_dp]
display(Markdown('### Left Disparity Map'))
display(triclopsi2_PIL[0])
display(Markdown('### Right Disparity Map'))
display(triclopsi2_PIL[1])

In [ ]:
occlude_dp = occlusion_check(triclopsi2_lcrpd, triclopsi2_rcrpd, triclopsi2_dp[0], numDisparities)
occlude_dp_filtered = cv.medianBlur(occlude_dp, 3)

display(Markdown('### Original Disparity Map'))
display(Image.fromarray(triclopsi2_dp[0]))
display(Markdown('### Occlusion Filling'))
display(Image.fromarray(occlude_dp))
display(Markdown('### Occlusion Filling + Median Filtering'))
display(Image.fromarray(occlude_dp_filtered))